In [175]:
import numpy as np
import requests
import os
from numpy.linalg import eig

# Working with the Kirchoff matrix of the Gaussian Network Model

## Downloading any PDB file

Let's first of all store the PDB id we want to download and the path where the file needs to be located:

In [2]:
pdb_id = '4NKK'
download_path = "./"

And now let's define an auxiliary function to download pdb files to be stored locally in our machine.

In [3]:
def fetch_pdb(pdb_id, download_path="./"):
        url = 'http://files.rcsb.org/download/{}.pdb'.format(pdb_id)
        try:
            res = requests.get(url, allow_redirects=True)
        except:
            print("Could not fetch pdb from {}".format(url))
            return 
        
        file_path = os.path.join(download_path, pdb_id + ".pdb")
        with open(file_path, "wb") as f:
            f.write(res.content)
        print(f"The {pdb_id} file was downloaded succesfully!")

Time to try it!

In [4]:
fetch_pdb(pdb_id, download_path=download_path)

The 4NKK file was downloaded succesfully!


## Parsing the positions of the CA atoms

We are going take an approach not very short and efficiente to obtain the coordinates of the CA atoms. But let's do it this way just once:

In [5]:
file_path = os.path.join(download_path, pdb_id + ".pdb")

CA_coordinates = []

with open(file_path) as fff:
    for line in fff.readlines():
        if line.startswith('ATOM  ') and line.split()[2]=='CA' :
          
            x, y, z = line.split()[6:9]
            CA_coordinates.append([float(x), float(y), float(z)])

Ok, `CA_coordinates` is a list of lists... let's convert it to a numpy array:

In [6]:
CA_coordinates = np.array(CA_coordinates)

In [244]:
#Alpha carbon number
len(CA_coordinates)

99

## Defining the Kirchhoff matrix

In [245]:
def kirchhoff_matrix(CA):
  A=np.zeros((len(CA),len(CA)))
  for i in range(len(A)):
    for j in range(len(A)):
      if i==j:
        continue
      r=CA[i]-CA[j]
      d=np.sqrt(np.dot(r,r))
      if d<9:
        A[i][j]=-1
        A[i][i]+=1
  return A

In [246]:
A=kirchhoff_matrix(CA_coordinates)
print(A)


[[ 3. -1. -1. ...  0.  0.  0.]
 [-1.  3. -1. ...  0.  0.  0.]
 [-1. -1.  5. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  5. -1. -1.]
 [ 0.  0.  0. ... -1.  3. -1.]
 [ 0.  0.  0. ... -1. -1.  2.]]


## Diagonalizing from its eigenvalues

In [247]:
#w:eigenvalues; v:eigenvectors
w,v=eig(A)

In [248]:
#Rounding eigenvalues to 10 digits
w=np.round(w,10)
print(w)

[20.78665754 20.42434018 19.8325264   0.          0.5197002   0.61588676
  0.83626185  1.92598286  1.64218979  1.68490315  2.40988656 18.90541643
 18.62715984  3.12577142  3.32861177 18.25020556  3.56399207  3.6836059
 18.05442935  3.93819502 17.91996179  4.22833998  4.54425632 17.4990182
 17.38037741 17.23177569  5.07668546 16.84745339 16.52112958 16.24969375
 16.15461876  5.29027057  5.27795016  5.67772533 15.84355079  6.19568691
 15.68236414  6.38061816  6.57593396 15.52585389 15.45582289  6.76338342
 15.26109589  6.89482929  7.02423358 15.21021425  7.22457248  7.46877614
 14.94816622 14.73071753 14.61923771 14.56759951  7.86151925  7.75890996
  8.02656396 14.26131348  8.17673328 14.18471007  8.32526219 13.97811834
  8.62638514  8.71933292  8.836277   13.74142793 13.69180872 13.50137948
  8.97048863 13.28858058 13.58429478 13.14137416 12.91097459 13.04719002
 12.79387767  9.0836881   9.24932475  9.41076216  9.50362432 12.49558515
 12.42928237 12.28374514  9.83279517  9.90529254 10.2

In [249]:
#sorting eigenvalues and eigenvectors in increasing order
idx = w.argsort()  
w = w[idx]
v = v[:,idx]

In [250]:
#Building diagonal matrix
D=np.diag(w)
print(D)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.5197002   0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.61588676 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ... 19.8325264   0.
   0.        ]
 [ 0.          0.          0.         ...  0.         20.42434018
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  20.78665754]]


## Verifying the system is diagonalized

In [251]:
F=np.dot(v,np.dot(D,(np.transpose(v))))
print(F)

[[ 3.00000000e+00 -1.00000000e+00 -1.00000000e+00 ...  1.44208864e-12
   5.74366509e-13  5.96286128e-12]
 [-1.00000000e+00  3.00000000e+00 -1.00000000e+00 ...  3.71067066e-12
   7.18041295e-13  3.92605382e-12]
 [-1.00000000e+00 -1.00000000e+00  5.00000000e+00 ... -1.11900177e-11
   4.68529339e-12  3.54063133e-12]
 ...
 [ 1.44206435e-12  3.71058739e-12 -1.11901322e-11 ...  5.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 5.74366076e-13  7.18041295e-13  4.68540354e-12 ... -1.00000000e+00
   3.00000000e+00 -1.00000000e+00]
 [ 5.96286117e-12  3.92605382e-12  3.54074235e-12 ... -1.00000000e+00
  -1.00000000e+00  2.00000000e+00]]


In [252]:
#Equal except for round-off errors
np.allclose(F,A)

True